# SAMueL Outcomes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../output/thrombolysis_choice_results.csv')
data = data.drop(columns=['Unnamed: 0'])

In [3]:
# Add columns
data['untreated_mrs_0-1'] = data['untreated_mrs_0'] + data['untreated_mrs_1']
data['untreated_mrs_0-2'] = data['untreated_mrs_0'] + data['untreated_mrs_1'] + data['untreated_mrs_2']
data['treated_mrs_0-1'] = data['treated_mrs_0'] + data['treated_mrs_1']
data['treated_mrs_0-2'] = data['treated_mrs_0'] + data['treated_mrs_1'] + data['treated_mrs_2']


## Run analysis of outcomes on ischaemic stroke

In [4]:
analyse_infarction_only = False

mask = (data['infarction'] == 1)
if analyse_infarction_only:
    data = data[mask]
iscahaemic = mask.mean()
print(f'Proportion of patients with infarction: {iscahaemic:0.4f}')


Proportion of patients with infarction: 0.8540


In [5]:
patient_groups = dict()

In [6]:
proportion_treated = []

# Get actually treated population
mask = data['thrombolysis'] == 1
patient_groups['actually treated'] = data[mask]
treated = mask.mean()
proportion_treated.append(treated)
print(f'Proportion of ischaemic patients treated: {treated:0.4f}')

# Get improved outcomes (including anticoagulant)
mask = data['improved_outcome'] == 1
patient_groups['improved_outcome including anticoagulant'] = data[mask]
improved_outcome = mask.mean()
proportion_treated.append(improved_outcome)
print(f'Proportion of ischaemic patients with improved outcome (including use of anticoagulants): {improved_outcome:0.4f}')

# Get improved outcomes (excluding anticoagulant)
mask = (data['improved_outcome'] == 1) & (data['afib_anticoagulant'] == 0)
patient_groups['improved_outcome_no_anticoagulant'] = data[mask]
improved_outcome_no_anticoagulant = mask.mean()
proportion_treated.append(improved_outcome_no_anticoagulant)
print(f'Proportion of ischaemic patients with improved outcome (excluding use of anticoagulants): {improved_outcome_no_anticoagulant:0.4f}')

# Get benchmark outcomes
mask = data['benchmark_decision'] == 1
patient_groups['benchmark'] = data[mask]
benchmark = mask.mean()
proportion_treated.append(benchmark)
print(f'Proportion of ischaemic patients with benchmark outcome: {benchmark:0.4f}')

Proportion of ischaemic patients treated: 0.3002
Proportion of ischaemic patients with improved outcome (including use of anticoagulants): 0.5847
Proportion of ischaemic patients with improved outcome (excluding use of anticoagulants): 0.5132
Proportion of ischaemic patients with benchmark outcome: 0.3690


In [7]:
# Results dataframe
results = pd.DataFrame(index=patient_groups.keys())
results['proportion_ischaemic_treated'] = proportion_treated

# Put summary results in lists
untreated_mrs_0_1 = []
treated_mrs_0_1 = []
change_in_mrs_0_1 = []
untreated_mrs_0_2 = []
treated_mrs_0_2 = []
change_in_mrs_0_2 = []
untreated_mrs6 = []
treated_mrs6 = []
change_in_mrs6 = []
untreated_weighted_mrs = []
treated_weighted_mrs = []
change_in_weighted_mrs = []
untreated_utility = []
treated_utility = []
change_in_utility = []


for key, group in patient_groups.items():
    # Patients in MRS 0-1
    untreated = np.mean(group['untreated_mrs_0'] + group['untreated_mrs_1'])
    treated = np.mean(group['treated_mrs_0'] + group['treated_mrs_1'])
    improved = treated - untreated
    untreated_mrs_0_1.append(untreated)
    treated_mrs_0_1.append(treated)
    change_in_mrs_0_1.append(improved)
    # Patients in MRS 0-2
    untreated = np.mean(group['untreated_mrs_0'] + group['untreated_mrs_1'] + group['untreated_mrs_2'])
    treated = np.mean(group['treated_mrs_0'] + group['treated_mrs_1'] + group['treated_mrs_2'])
    improved = treated - untreated
    untreated_mrs_0_2.append(untreated)
    treated_mrs_0_2.append(treated)
    change_in_mrs_0_2.append(improved)
    # Patients in MRS 6 (dead)
    untreated = np.mean(group['untreated_mrs_6'])
    treated = np.mean(group['treated_mrs_6'])
    improved = treated - untreated
    untreated_mrs6.append(untreated)
    treated_mrs6.append(treated)
    change_in_mrs6.append(improved)
    # Weighted MRS
    untreated = np.mean(group['untreated_weighted_mrs'])
    treated = np.mean(group['treated_weighted_mrs'])
    improved = treated - untreated
    untreated_weighted_mrs.append(untreated)
    treated_weighted_mrs.append(treated)
    change_in_weighted_mrs.append(improved)
    # Utility
    untreated = np.mean(group['untreated_utility'])
    treated = np.mean(group['treated_utility'])
    improved = treated - untreated
    untreated_utility.append(untreated)
    treated_utility.append(treated)
    change_in_utility.append(improved)

results['untreated_mrs_0_1'] = untreated_mrs_0_1
results['treated_mrs_0_1'] = treated_mrs_0_1
results['change_in_mrs_0_1'] = change_in_mrs_0_1
results['untreated_mrs_0_2'] = untreated_mrs_0_2
results['treated_mrs_0_2'] = treated_mrs_0_2
results['change_in_mrs_0_2'] = change_in_mrs_0_2
results['untreated_mrs6'] = untreated_mrs6
results['treated_mrs6'] = treated_mrs6
results['change_in_mrs6'] = change_in_mrs6
results['untreated_weighted_mrs'] = untreated_weighted_mrs
results['treated_weighted_mrs'] = treated_weighted_mrs
results['change_in_weighted_mrs'] = change_in_weighted_mrs
results['untreated_utility'] = untreated_utility
results['treated_utility'] = treated_utility
results['change_in_utility'] = change_in_utility

results.to_csv('./output/thrombolysis_benefit_summary.csv')


In [8]:
results.round(3).T

,actually treated,improved_outcome including anticoagulant,improved_outcome_no_anticoagulant,benchmark
proportion_ischaemic_treated,0.300,0.585,0.513,0.369
untreated_mrs_0_1,0.259,0.241,0.250,0.252
treated_mrs_0_1,0.356,0.322,0.335,0.352
change_in_mrs_0_1,0.097,0.082,0.085,0.099
untreated_mrs_0_2,0.417,0.394,0.406,0.411
treated_mrs_0_2,0.538,0.499,0.514,0.534
change_in_mrs_0_2,0.121,0.106,0.108,0.123
untreated_mrs6,0.222,0.229,0.220,0.225
treated_mrs6,0.143,0.155,0.146,0.144
change_in_mrs6,-0.080,-0.074,-0.073,-0.081


In [9]:
# Additional good outcomes per 1000 arrivals (4 hours)

summary = pd.DataFrame()

summary['additional mrs 0-1 per 1000 arrivals'] = (
    results['proportion_ischaemic_treated'] * results['change_in_mrs_0_1'] * 1000)

summary['additional mrs 0-2 per 1000 arrivals'] = (
    results['proportion_ischaemic_treated'] * results['change_in_mrs_0_2'] * 1000)

summary['mean change in weighted mRS'] = (
    results['proportion_ischaemic_treated'] * results['change_in_weighted_mrs'])

summary['mean change in utility'] = (
    results['proportion_ischaemic_treated'] * results['change_in_utility'])

summary['mean change in death'] = (
    results['proportion_ischaemic_treated'] * results['change_in_mrs6'])

summary = summary.round(3)

In [10]:
summary.T

,actually treated,improved_outcome including anticoagulant,improved_outcome_no_anticoagulant,benchmark
additional mrs 0-1 per 1000 arrivals,29.121,47.751,43.597,36.540
additional mrs 0-2 per 1000 arrivals,36.438,61.739,55.265,45.391
mean change in weighted mRS,-0.171,-0.299,-0.266,-0.214
mean change in utility,0.032,0.057,0.050,0.040
mean change in death,-0.024,-0.043,-0.038,-0.030


In [11]:
# What proportion of patients on anticoagulants are treated with thrombolysis?

mask = (data['afib_anticoagulant'] == 1)
selected_data = data[mask]
proportion_treated = selected_data['thrombolysis'].mean()
print(f'Proportion of patients on anticoagulants treated with thrombolysis: {proportion_treated:0.4f}')
proportion_treated = selected_data['benchmark_decision'].mean()
print(f'Proportion of patients on anticoagulants with benchmark decision: {proportion_treated:0.4f}')


Proportion of patients on anticoagulants treated with thrombolysis: 0.0716
Proportion of patients on anticoagulants with benchmark decision: 0.0189


In [12]:
# Proportion of benchmark decision that are improved outcomes
mask = (data['benchmark_decision'] == 1)
selected_data = data[mask]
proportion_improved = selected_data['improved_outcome'].mean()
print(f'Proportion of benchmark decisions that are improved outcomes: {proportion_improved:0.4f}')


Proportion of benchmark decisions that are improved outcomes: 0.9177


In [13]:
# Proportion of actual thrombolysis that are improved outcomes
mask = (data['thrombolysis'] == 1)
selected_data = data[mask]
proportion_improved = selected_data['improved_outcome'].mean()
print(f'Proportion of actual thrombolysis that are improved outcomes: {proportion_improved:0.4f}')


Proportion of actual thrombolysis that are improved outcomes: 0.8913


In [14]:
# Proportion of improved outcomes (no anticoagulation) that are benchmark decisions
mask = (data['improved_outcome'] == 1) & (data['afib_anticoagulant'] == 0)
selected_data = data[mask]
proportion_benchmark = selected_data['benchmark_decision'].mean()
print(f'Proportion of improved outcomes (no anticoagulation) that are benchmark decisions: {proportion_benchmark:0.4f}')

Proportion of improved outcomes (no anticoagulation) that are benchmark decisions: 0.6548


In [15]:
# Proportion of improved outcomes (no anticoagulation) that are treated with thrombolysis
mask = (data['improved_outcome'] == 1) & (data['afib_anticoagulant'] == 0)
selected_data = data[mask]
proportion_treated = selected_data['thrombolysis'].mean()
print(f'Proportion of improved outcomes (no anticoagulation) treated with thrombolysis: {proportion_treated:0.4f}')

Proportion of improved outcomes (no anticoagulation) treated with thrombolysis: 0.5041


In [16]:
data = pd.read_csv('../output/thrombolysis_choice_results.csv')
data = data.drop(columns=['Unnamed: 0'])

mask = (data['thrombolysis'] == 0) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("No thrombolysis, no improved outcome: ", total)
mask = (data['thrombolysis'] == 0) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("No thrombolysis, improved outcome: ", total)
mask = (data['thrombolysis'] == 1) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("Thrombolysis, no improved outcome: ", total)
mask = (data['thrombolysis'] == 1) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("Thrombolysis, improved outcome: ", total)


No thrombolysis, no improved outcome:  50681
No thrombolysis, improved outcome:  41989
Thrombolysis, no improved outcome:  4320
Thrombolysis, improved outcome:  35436


In [17]:
mask = (data['benchmark_decision'] == 0) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("No thrombolysis, no improved outcome: ", total)
mask = (data['benchmark_decision'] == 0) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("No thrombolysis, improved outcome: ", total)
mask = (data['benchmark_decision'] == 1) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("Thrombolysis, no improved outcome: ", total)
mask = (data['benchmark_decision'] == 1) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("Thrombolysis, improved outcome: ", total)

No thrombolysis, no improved outcome:  50978
No thrombolysis, improved outcome:  32585
Thrombolysis, no improved outcome:  4023
Thrombolysis, improved outcome:  44840


In [18]:
sensitivity = data['TP'].sum() / (data['TP'].sum() + data['FN'].sum())
specificity = data['TN'].sum() / (data['TN'].sum() + data['FP'].sum())

print(f'Sensitivity: {sensitivity:0.4f}')
print(f'Specificity: {specificity:0.4f}')


Sensitivity: 0.4577
Specificity: 0.8789
